In [1]:
import numpy as np
import pandas as pd
import LatentFactor
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from LatentFactor import RecommendList
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,recall_score,roc_curve,precision_score
%matplotlib inline

In [2]:
dftrain=pd.read_csv('train.csv')
dftest=pd.read_csv('test.csv')
item_pool=pd.read_csv('item_pool.csv')
item_pool=list(item_pool['oper_obj'])

In [3]:
#training Latent factor model, which generates embedding vectors for users and items.
lf=LatentFactor.LatentFactor(F=32,item_pool=item_pool,neg_ratio=2,iters=50,eta=0.08,lamda=0.01,verbose=True)
lf.fit(dftrain)

===============Iteration 0===============

cross entropy loss: 0.643332014857206
===============Iteration 1===============

cross entropy loss: 0.566110612826628
===============Iteration 2===============

cross entropy loss: 0.4938021388316874
===============Iteration 3===============

cross entropy loss: 0.43558875511234085
===============Iteration 4===============

cross entropy loss: 0.38465554419698944
===============Iteration 5===============

cross entropy loss: 0.33741213912079737
===============Iteration 6===============

cross entropy loss: 0.2937469938898363
===============Iteration 7===============

cross entropy loss: 0.2570077921109922
===============Iteration 8===============

cross entropy loss: 0.22934136849541437
===============Iteration 9===============

cross entropy loss: 0.20906712218257298
===============Iteration 10===============

cross entropy loss: 0.19489804453754112
===============Iteration 11===============

cross entropy loss: 0.18310268754942188
=========

In [4]:
train=lf.getDataset(dftrain,2)
test=lf.getDataset(dftest,2)
lgb_cols=[col for col in train.columns if col not in ['user_id','item_id','label']]
lgbtrain=lgb.Dataset(train[lgb_cols],label=train['label'])
lgbtest=lgb.Dataset(test[lgb_cols],label=test['label'])
lgb_rank_X_train=train.sort_values(by=['user_id']).reset_index(drop=True)
lgb_rank_X_test=test.sort_values(by=['user_id']).reset_index(drop=True)
group_train=train.sort_values(by=['user_id']).groupby(['user_id'],as_index=False).count()['label'].values
group_test=test.sort_values(by=['user_id']).groupby(['user_id'],as_index=False).count()['label'].values
del train
del test

In [19]:
#Lightgbm ranking model
lgb_ranker=lgb.LGBMRanker(boosting_type='goss',n_estimators=400,num_leaves=8,min_child_weight=0.001,reg_alpha=0.1,reg_lambda=0.08,max_depth=6,subsample=0.6,colsample_bytree=0.6,learning_rate=0.015)
lgb_ranker.fit(lgb_rank_X_train[lgb_cols],lgb_rank_X_train['label'],group=group_train,eval_set=[(lgb_rank_X_test[lgb_cols],lgb_rank_X_test['label'])],eval_names=['test'],eval_metric='ndcg',eval_at=[1,2,3,4,5,6,7],eval_group=[group_test],early_stopping_rounds=100,verbose=20)

Training until validation scores don't improve for 100 rounds
[20]	test's ndcg@1: 0.309251	test's ndcg@2: 0.382827	test's ndcg@3: 0.448335	test's ndcg@4: 0.495412	test's ndcg@5: 0.526073	test's ndcg@6: 0.548468	test's ndcg@7: 0.566088
[40]	test's ndcg@1: 0.309251	test's ndcg@2: 0.384279	test's ndcg@3: 0.449851	test's ndcg@4: 0.496267	test's ndcg@5: 0.52635	test's ndcg@6: 0.549742	test's ndcg@7: 0.566935
[60]	test's ndcg@1: 0.367401	test's ndcg@2: 0.399735	test's ndcg@3: 0.461249	test's ndcg@4: 0.506674	test's ndcg@5: 0.536048	test's ndcg@6: 0.559053	test's ndcg@7: 0.576003
[80]	test's ndcg@1: 0.346256	test's ndcg@2: 0.39592	test's ndcg@3: 0.463259	test's ndcg@4: 0.512747	test's ndcg@5: 0.538868	test's ndcg@6: 0.558341	test's ndcg@7: 0.575476
[100]	test's ndcg@1: 0.365639	test's ndcg@2: 0.418853	test's ndcg@3: 0.483342	test's ndcg@4: 0.530455	test's ndcg@5: 0.554382	test's ndcg@6: 0.574638	test's ndcg@7: 0.589933
[120]	test's ndcg@1: 0.390308	test's ndcg@2: 0.44939	test's ndcg@3: 0.5060

LGBMRanker(boosting_type='goss', class_weight=None, colsample_bytree=0.6,
           importance_type='split', learning_rate=0.015, max_depth=6,
           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
           n_estimators=400, n_jobs=-1, num_leaves=8, objective=None,
           random_state=None, reg_alpha=0.1, reg_lambda=0.08, silent=True,
           subsample=0.6, subsample_for_bin=200000, subsample_freq=0)

In [26]:
#lightgbm binary classification
eval_result={}
clf_params={'objective':'binary','metric':'auc','boosting':'goss','eta':0.05,'num_leaves':12,'max_depth':6,
        'min_sum_hessian_in_leaf':0.002,'subsample':0.6,'colsample_bytree':0.6,'reg_alpha':0.2,'reg_lambda':0.08,'is_unbalance':True}
lgb_clf=lgb.train(clf_params,lgbtrain,num_boost_round=1500,early_stopping_rounds=100,valid_sets=[lgbtrain,lgbtest],valid_names=['train','test'],verbose_eval=200,evals_result=eval_result)  

[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.008, min_child_weight=0.006 will be ignored. Current value: min_sum_hessian_in_leaf=0.008
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.008, min_child_weight=0.006 will be ignored. Current value: min_sum_hessian_in_leaf=0.008
[LightGBM] [Info] Number of positive: 110949, number of negative: 150462
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050402 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11072
[LightGBM] [Info] Number of data points in the train set: 261411, number of used features: 64
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.424424 -> initscore=-0.304640
[LightGBM] [Info] Start training from score -0.304640
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.935222	test's auc: 0.6054
[400]	train's auc: 0.969951	test's auc: 0.618107
[600]	train's auc: 0.980

In [27]:
#Predictions and the ground trueth
result_rank=RecommendList(lgb_rank_X_test,lgb_ranker,lgb_cols)
result_clf=RecommendList(lgb_rank_X_test,lgb_clf,lgb_cols)
result_true=RecommendList(lgb_rank_X_test,lgb_ranker,lgb_cols,is_predict=False)

In [28]:
#Performance
rank_pred=result_rank.melt(id_vars=['user_id']).dropna().drop(columns='variable').rename(columns={'user_id':'agent_id','value':'oper_obj'})
clf_pred=result_clf.melt(id_vars=['user_id']).dropna().drop(columns='variable').rename(columns={'user_id':'agent_id','value':'oper_obj'})
print(lf.Recall(dftest,rank_pred))
print(lf.Recall(dftest,clf_pred))

0.25219717233473443
0.30855941918226976
